In [285]:
!pip3 install yfinance ta

In [286]:
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date


### Question 1: [Macro] Average growth of GDP in 2023

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1).
Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers).
Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer  5.7

In [287]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-5, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=21
Period for indexes: 2019-04-21 to 2024-04-21 


In [288]:
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)
gdpc1['gdpc1_yoy'] = gdpc1.GDPC1/gdpc1.GDPC1.shift(4)-1
gdpc1


,GDPC1,gdpc1_yoy
DATE,,
2019-07-01,20817.581,NaN
2019-10-01,20951.088,NaN
2020-01-01,20665.553,NaN
2020-04-01,19034.830,NaN
2020-07-01,20511.785,-0.014689
2020-10-01,20724.128,-0.010833
2021-01-01,20990.541,0.015726
2021-04-01,21309.544,0.119503
2021-07-01,21483.083,0.047353


In [289]:
df_q2023 = gdpc1.loc['2023-01-01':'2023-10-01']
avg_2023 = df_q2023.gdpc1_yoy.mean()
f"average 2023 yoy growth is {round(avg_2023*100,1)}%"



'average 2023 yoy growth is 2.5%'

### Question 2. [Macro] Inverse "Treasury Yield"

**Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**


Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2,
 https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)


In [290]:
import pandas as pd

dgs2 = pdr.DataReader("DGS2", "fred", start=start)
dgs10 = pdr.DataReader("DGS10", "fred", start=start)
start_date = '2020-01-01'
last_date = dgs2.index[-1].date().strftime('%Y-%m-%d')
dgs2_2020, dgs10_2020 = dgs2.loc[f"{start_date}":f"{last_date}"], dgs10.loc[f"{start_date}":f"{last_date}"]

combined_df = dgs2_2020.join(dgs10_2020)
combined_df['DGS10 - DGS2'] = combined_df['DGS10'] - combined_df['DGS2']
f"min difference beween dgs10 and dgs2 is {round(combined_df['DGS10 - DGS2'].min(),1)}"







'min difference beween dgs10 and dgs2 is -1.1'

In [310]:
combined_df

,DGS2,DGS10,DGS10 - DGS2
DATE,,,
2020-01-01,NaN,NaN,NaN
2020-01-02,1.58,1.88,0.30
2020-01-03,1.53,1.80,0.27
2020-01-06,1.54,1.81,0.27
2020-01-07,1.54,1.83,0.29
...,...,...,...
2024-04-12,4.88,4.50,-0.38
2024-04-15,4.93,4.63,-0.30
2024-04-16,4.97,4.67,-0.30


When the yield curve inverts, meaning that the yield on shorter-term bonds (like the 2-year Treasury securities) exceeds the yield on longer-term bonds (such as the 10-year Treasury securities), it often signifies a pessimistic outlook on the economy.

An inverted yield curve is a leading indicator of an impending economic downturn, lower interest rates in the future, lower corporate revenues and reduced consumer spending. This would definitely be a good feature for a model as this would aid traders to implement risk management strategies that would protect their portfilio against market declines and enable them choose sectors and stocks that would perform better in a recession.

### Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)


In [291]:
gspc_daily = yf.download(tickers = "^GSPC",
                     period = "max",
                     interval = "1d")
mxx_daily = yf.download(tickers = "^MXX",
                     period = "max",
                     interval = "1d")
gspc_5y, mxx_5y = gspc_daily.loc['2018-04-09':'2024-04-09'], mxx_daily.loc['2018-04-09':'2024-04-09']
gspc_5y



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-04-09,2617.179932,2653.550049,2610.790039,2613.159912,2613.159912,3076140000
2018-04-10,2638.409912,2665.449951,2635.780029,2656.870117,2656.870117,3579130000
2018-04-11,2643.889893,2661.429932,2639.250000,2642.189941,2642.189941,3054520000
2018-04-12,2653.830078,2674.719971,2653.830078,2663.989990,2663.989990,3045330000
2018-04-13,2676.899902,2680.260010,2645.050049,2656.300049,2656.300049,2988540000
...,...,...,...,...,...,...
2024-04-03,5194.370117,5228.750000,5194.370117,5211.490234,5211.490234,3703250000
2024-04-04,5244.049805,5256.589844,5146.060059,5147.209961,5147.209961,4075680000
2024-04-05,5158.950195,5222.180176,5157.209961,5204.339844,5204.339844,3386780000


In [292]:

num = 365
index1 = {
  "name_index" : "gspc",
  "num_days_in_year" : num,
  "df_name": "gspc_daily"
}
index2 = {
  "name_index" : "mxx",
  "num_days_in_year" : num,
  "df_name": "mxx_daily"
}

indices = {
  "index1" : index1,
  "index2" : index2
}
for k, v in indices.items():
  params = (indices[k]["name_index"], "growth", str(indices[k]["num_days_in_year"]))
  col_name = "_".join(params)
  if indices[k]["df_name"] == "gspc_daily":
    gspc_5y_copy = gspc_5y.copy(deep=True)
    gspc_5y_copy[col_name] = gspc_5y_copy.Close/gspc_5y_copy.Close.shift(365)
    gspc = gspc_5y_copy.add_suffix("_gspc")
  if indices[k]["df_name"] == "mxx_daily":
    mxx_5y_copy = mxx_5y.copy(deep=True)
    #convert close price for mxx index from mexican pesos to USD
    mxx_5y_copy['Close_in_dollars'] = mxx_5y_copy.Close*0.058
    mxx_5y_copy[col_name] = mxx_5y_copy.Close_in_dollars/mxx_5y_copy.Close_in_dollars.shift(365)
    mxx = mxx_5y_copy.add_suffix("_mxx")

In [293]:

combined_df_i = gspc.join(mxx)
columns = ['Close_gspc', 'Close_in_dollars_mxx']
df_growth_365 = pd.DataFrame(combined_df_i, columns=columns)
df_growth_365 = df_growth_365.filter(items = ['2019-04-09', '2024-04-09'], axis=0)
df_growth_365





,Close_gspc,Close_in_dollars_mxx
Date,,
2019-04-09,2878.199951,2618.794477
2024-04-09,5209.910156,3339.126609


In [294]:
growth_gsps, growth_xx = (df_growth_365.iloc[1]['Close_gspc'] - df_growth_365.iloc[0]['Close_gspc'])/df_growth_365.iloc[0]['Close_gspc'], (df_growth_365.iloc[1]['Close_in_dollars_mxx'] - df_growth_365.iloc[0]['Close_in_dollars_mxx'])/df_growth_365.iloc[0]['Close_in_dollars_mxx']
if growth_gsps >= growth_xx:
  print(f"S&P 500 index has {round(growth_gsps*100,0)}% increase in growth from 2019 - 2024 compared to MXX index of {round(growth_xx*100,0)}%")
else:
  print(f"MXX index has {round(growth_xx*100,0)}% increase in growth from 2019 - 2024 compared to S&P 500 index of {round(growth_gsps*100,0)}%")

S&P 500 index has 81.0% increase in growth from 2019 - 2024 compared to MXX index of 28.0%


### Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks


**Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023**


Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value.
Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [304]:
def minmax(stock, start=start, end=end):
  df = yf.download(tickers = stock,
             period = "max",
             interval = "1d")
  daily_df_close = df.loc[start:end]
  df_copy = daily_df_close.copy(deep=True)
  min_max = (df_copy['Adj Close'].max() - df_copy['Adj Close'].min())/df_copy['Adj Close'].max()
  return min_max




In [305]:

start = '2023-01-01'
end = '2023-12-31'
list_of_stocks = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]
minmax_dict = {}
for stock in list_of_stocks:
  if stock == "2222.SR":
    min_max_saudi = minmax(stock)
    minmax_dict[stock] = min_max_saudi
  elif stock == "BRK-B":
    min_max_bs = minmax(stock)
    minmax_dict[stock] = min_max_bs
  elif stock == "AAPL":
    min_max_apple = minmax(stock)
    minmax_dict[stock] = min_max_apple
  elif stock == "MSFT":
    min_max_msft = minmax(stock)
    minmax_dict[stock] = min_max_msft
  elif stock == "GOOG":
    min_max_google = minmax(stock)
    minmax_dict[stock] = min_max_google
  else:
    min_max_jpm = minmax(stock)
    minmax_dict[stock] = min_max_jpm

max_v = max(zip(minmax_dict.values(), minmax_dict.keys()))
f"Company with max value:{max_v[1]} and value:{round(max_v[0],2)}"




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


'Company with max value:MSFT and value:0.42'

### Question 5. [Stocks] Dividend Yield
**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
You can use `get_actions()` method or `.dividends` field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)


In [297]:

def get_sum_div(stock, start=start,end=end):
  div = yf.Ticker(stock).get_actions().loc[start:end]
  sum = div['Dividends'].sum()
  return sum

def last_price(stock, start=start, end=end):
  df = yf.download(tickers = stock,
             period = "max",
             interval = "1d")
  daily_df_close = df.loc[start:end]
  df_copy = daily_df_close.copy(deep=True)
  last_value = df_copy.iloc[-1]['Adj Close']
  return last_value

def extract_div(stock):
  sum_div = get_sum_div(stock)
  last_price_stock = last_price(stock)
  div_yield = round(sum_div/last_price_stock*100,1)
  print(f"{stock}: 2023 total dividends paid {sum_div} and the end year 2023 stock price is {last_price_stock}, the dividend yield is {div_yield}")
  return div_yield



In [298]:

sumdiv = {}
for stock in list_of_stocks:
  if stock == "MSFT":
    div_yield_msft = extract_div(stock)
    sumdiv[stock] = div_yield_msft
  elif stock == "2222.SR":
    div_yield_saudi = extract_div(stock)
    sumdiv[stock] = div_yield_saudi
  elif stock == "BRK-B":
    div_yield_bs = extract_div(stock)
    sumdiv[stock] = div_yield_bs
  elif stock == "AAPL":
    div_yield_apple = extract_div(stock)
    sumdiv[stock] = div_yield_apple
  elif stock == "GOOG":
    div_yield_google = extract_div(stock)
    sumdiv[stock] = div_yield_google
  else:
    div_yield_jpm = extract_div(stock)
    sumdiv[stock] = div_yield_jpm


max_v = max(zip(sumdiv.values(), sumdiv.keys()))
f"Company with max value:{max_v[1]} and value:{max_v[0]}"



[*********************100%%**********************]  1 of 1 completed


2222.SR: 2023 total dividends paid 0.9107640000000001 and the end year 2023 stock price is 32.82804870605469, the dividend yield is 2.8


[*********************100%%**********************]  1 of 1 completed


BRK-B: 2023 total dividends paid 0.0 and the end year 2023 stock price is 356.6600036621094, the dividend yield is 0.0


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


GOOG: 2023 total dividends paid 0.0 and the end year 2023 stock price is 140.92999267578125, the dividend yield is 0.0


[*********************100%%**********************]  1 of 1 completed

JPM: 2023 total dividends paid 4.05 and the end year 2023 stock price is 168.07713317871094, the dividend yield is 2.4


'Company with max value:2222.SR and value:2.8'

### Question 6. [Exploratory] Investigate new metrics

**Free text answer**

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

In [306]:
def label_rsi(value):
  if value >= 70:
    return 'bearish'
  elif value < 30:
    return 'bullish'
  else:
    return 'neutral'

def compare_price(value):
  if value > 0:
    return 'uptrend'
  else:
    return 'downtrend'



In [309]:
import ta

start = '2018-04-22'
end = '2024-04-22'
stock = "NVDA"
index = "^IXIC"
df = yf.download(tickers = stock,
             period = "max",
             interval = "1d")
df_i = yf.download(tickers = index,
             period = "max",
             interval = "1d")
df_nv = df.loc[start:end]
df_i_nasdaq = df_i.loc[start:end]
df_nv_copy = df_nv.copy(deep=True)
df_i_copy = df_i_nasdaq.copy(deep=True)
df_i_copy['nasdaq_close'] = df_i_copy['Close']
df_i_close = df_i_copy['nasdaq_close']
df_nv_copy = df_nv_copy.join(df_i_close)
df_nv_copy['correlation_with_nasdaq_index'] = df_nv_copy['Close'].corr(df_nv_copy['nasdaq_close'])
df_nv_copy['previous_close'] = df_nv_copy.Close.shift(1)
df_nv_copy['dod'] = (df_nv_copy.Close - df_nv_copy.Close.shift(1))/df_nv_copy.Close*100
df_nv_copy['bb_high'], df_nv_copy['bb_mid'], df_nv_copy['bb_low'] = ta.volatility.bollinger_hband(df_nv_copy['Close']), ta.volatility.bollinger_mavg(df_nv_copy['Close']), ta.volatility.bollinger_lband(df_nv_copy['Close'])
df_nv_copy['historical_average_price_20'] = ta.volatility.bollinger_mavg(df_nv_copy['Close'], window=20)
df_nv_copy['comparison_to_historical_price'] = df_nv_copy.Close - df_nv_copy.historical_average_price_20
df_nv_copy['comparison_price_labels'] = df_nv_copy['comparison_to_historical_price'].apply(compare_price)
df_nv_copy['rsi'] = ta.momentum.rsi(close=df_nv_copy['Close'])
df_nv_copy['rsi_labels'] = df_nv_copy['rsi'].apply(label_rsi)
df_nv_copy.head(50)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,nasdaq_close,correlation_with_nasdaq_index,previous_close,dod,bb_high,bb_mid,bb_low,historical_average_price_20,comparison_to_historical_price,comparison_price_labels,rsi,rsi_labels
Date,,,,,,,,,,,,,,,,,,
2018-04-23,57.357498,57.875000,55.707500,55.970001,55.412540,42750800,7128.600098,0.754337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-04-24,56.230000,57.200001,54.639999,55.299999,54.749218,41459600,7007.350098,0.754337,55.970001,-1.211577,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-04-25,55.000000,55.395000,52.575001,54.165001,53.625519,58849200,7003.740234,0.754337,55.299999,-2.095446,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-04-26,55.799999,56.387501,55.132500,56.305000,55.744202,37582400,7118.680176,0.754337,54.165001,3.800727,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-04-27,57.375000,57.575001,56.154999,56.582500,56.018940,40084800,7119.799805,0.754337,56.305000,0.490435,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-04-30,56.747501,57.250000,56.029999,56.224998,55.664993,32784400,7066.270020,0.754337,56.582500,-0.635842,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-05-01,56.142502,56.812500,55.549999,56.785000,56.219418,25378800,7130.700195,0.754337,56.224998,0.986178,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-05-02,56.750000,57.200001,56.312500,56.577499,56.013988,26824800,7100.899902,0.754337,56.785000,-0.366754,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral
2018-05-03,56.900002,58.512501,56.442501,58.247501,57.667366,44314400,7088.149902,0.754337,56.577499,2.867079,NaN,NaN,NaN,NaN,NaN,downtrend,NaN,neutral





*   correlation_with_nasdaq_index: shows strength of relationship between close price of stock price and close price of industry index(in this case of NVIDIA, it is the Nasdaq index). A strong positive correlation shows when index price increases, stock price will increase
*   dod: percentage change in closing price from the previous day. Positive dod shows increase in closing price from previous day, negative dod shows decrease in closing price from previous day and dod values close to zero show little or no change
*   bb_high, bb_mid, bb_low: stock price goes above bb_high value, bearish signal and potential correction; stock price falls below bb_low value, bullish signal and potential rally and bb_mid value is a reference point for the average stock price
*   comparison of historical average closing price (over 20 days) to closing price can seen as a lagging indicator and a way to confirm a trend.If current closing price is greater than historical average, then it's a bullish signal indicating an uptrend, if current closing price is lower than historical signal, then it's a bearish signal indicating a downtrend. This would indicate more of a macro trend
*   rsi and rsi labels: RSI is a leading indicator and can alert the trader of what is to come. RSI > 70 is a bearish signal indicating that the market is overbought, price correction on the horizon and therefore sell signal. An rsi < 30 is a bullish signal, indicating the market is oversold, a potential rally and a buy signal. Any value between 30 and 70 is a potential sideways market where it's neither overbought or oversold
*   Fundamental indicators that provide a picture of the overall health of economy include interest rates, consumer price index and unemployment figures











### Question 7. [Exploratory] Time-driven strategy description around earnings releases

**Free text answer**

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08).

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.





*   Scrape url: [past eps](https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08&offset=0&sf=ticker&st=desc) using beautiful soup python library to extract date,        
 ticker_symbol, reported_EPS
*   Scrape url: [future eps](https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23) using beautiful soup python library to extract date, ticker_symbol, estimated_EPS

*   Create a past_eps dataframe scraping the above past_eps data for each date

*   Create a future_eps dataframe scraping the above future_eps data for each date

*   Merge the two datasets with date as the index
*   Calculate EPS growth rate field from past eps dates to future eps dates
*   Filter for positive EPS growth values (value > 0) and this will provide a basic subset of companies that show potential for growth and can be investigated further









